In [ ]:
!pip install transformers

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')



In [ ]:

# Define a custom embedding layer
class CustomEmbedding(nn.Module):
    def __init__(self, embedding_size):
        super(CustomEmbedding, self).__init__()
        self.embedding_size = embedding_size
        self.embedding = nn.Embedding(tokenizer.vocab_size, embedding_size)
        
    def forward(self, input_ids, token_weights):
        # Get the token embeddings from the default BERT model
        embeddings = self.embedding(input_ids)
        
        # Modify the embeddings based on the token weights
        modified_embeddings = embeddings * token_weights.unsqueeze(-1)
        
        return modified_embeddings

# Example sentence
sentence = "This is an example sentence."

# Tokenize the sentence
input_ids = torch.tensor(tokenizer.encode(sentence, add_special_tokens=True)).unsqueeze(0)

# Define the token weights
token_weights = torch.tensor([1, 1, 2, 3, 1, 1, 1, 1]).float()

# Define the custom embedding layer with an embedding size of 768 (default for BERT)
custom_embedding = CustomEmbedding(768)

# Get the modified token embeddings from the custom embedding layer
modified_embeddings = custom_embedding(input_ids, token_weights)

# Pass the modified token embeddings to the BERT model for further processing
output = bert_model(inputs_embeds=modified_embeddings)

# Get the final output from the BERT model
pooled_output = output[1]

### Performance evaluation for two sentence without adjusting weights

In [ ]:
sentence1 = "This is stomach related issue"
sentence2 = "I have pain in stomach"

In [ ]:
# Tokenize the sentence
input_ids_1 = torch.tensor(tokenizer.encode(sentence1, add_special_tokens=True)).unsqueeze(0)

# Define the token weights
token_weights_1 = torch.tensor([1]*input_ids_1.shape[1]).float()

# Define the custom embedding layer with an embedding size of 768 (default for BERT)
custom_embedding_1 = CustomEmbedding(768)

# Get the modified token embeddings from the custom embedding layer
modified_embeddings_1 = custom_embedding_1(input_ids_1, token_weights_1)

# Pass the modified token embeddings to the BERT model for further processing
output_1 = bert_model(inputs_embeds=modified_embeddings_1)

# Get the final output from the BERT model
pooled_output_1 = output_1[1]

In [ ]:
# Tokenize the sentence
input_ids_2 = torch.tensor(tokenizer.encode(sentence2, add_special_tokens=True)).unsqueeze(0)

# Define the token weights
token_weights_2 = torch.tensor([1]*input_ids_2.shape[1]).float()

# Define the custom embedding layer with an embedding size of 768 (default for BERT)
custom_embedding_2 = CustomEmbedding(768)

# Get the modified token embeddings from the custom embedding layer
modified_embeddings_2 = custom_embedding_2(input_ids_2, token_weights_2)

# Pass the modified token embeddings to the BERT model for further processing
output_2 = bert_model(inputs_embeds=modified_embeddings_2)

# Get the final output from the BERT model
pooled_output_2 = output_2[1]

In [ ]:
import torch.nn.functional as F

# Calculate the cosine similarity between the two pooled outputs
cosine_sim = F.cosine_similarity(pooled_output_1, pooled_output_2)

# Print the cosine similarity
print("Cosine similarity between the two sentences:", cosine_sim.item())

Cosine similarity between the two sentences: 0.8298619985580444


### Check performance with SBERT for comparison

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util

sentence1 = "This is stomach related issue"
sentence2 = "I have pain in stomach"

# Load pre-trained SBERT model
model = SentenceTransformer('bert-base-nli-mean-tokens')

# Get sentence embeddings
embeddings_1 = model.encode(sentence1, convert_to_tensor=True)
embeddings_2 = model.encode(sentence2, convert_to_tensor=True)

# Calculate cosine similarity between the sentence embeddings
cosine_sim = util.pytorch_cos_sim(embeddings_1, embeddings_2)

# Print the cosine similarity
print("Cosine similarity between the two sentences:", cosine_sim.item())

Cosine similarity between the two sentences: 0.7642334699630737


In [ ]:
from sentence_transformers import SentenceTransformer, util

# Load pre-trained SBERT model
model = SentenceTransformer('bert-base-nli-mean-tokens')

sentence1 = "This is stomach related issue"
sentence2 = "I have pain in stomach"

# Get sentence embeddings
embeddings_1 = model.encode(sentence1, convert_to_tensor=True)
embeddings_2 = model.encode(sentence2, convert_to_tensor=True)


token_weights = torch.tensor([1]*embeddings_1.shape[0]).float()
modified_embeddings_1 = embeddings_1 * token_weights.unsqueeze(-1)

modified_embeddings_1 = embeddings_2 * token_weights.unsqueeze(-1)

# Calculate cosine similarity between the sentence embeddings
cosine_sim = util.pytorch_cos_sim(embeddings_1, embeddings_2)

# Print the cosine similarity
print("Cosine similarity between the two sentences:", cosine_sim.item())

Cosine similarity between the two sentences: 0.7642334699630737


### Performance evaluation for two sentence after adjusting weights

In [ ]:
# Tokenize the sentence
input_ids_1 = torch.tensor(tokenizer.encode(sentence1, add_special_tokens=True)).unsqueeze(0)

### We will be using 4th sentence and 6th item respectively for weightage
sentence1 = "This is stomach related issue"
sentence2 = "I have pain in stomach"


# Define the token weights
token_weights_1 = torch.tensor([1,1,1,5,1,1,1]).float()

# Define the custom embedding layer with an embedding size of 768 (default for BERT)
custom_embedding_1 = CustomEmbedding(768)

# Get the modified token embeddings from the custom embedding layer
modified_embeddings_1 = custom_embedding_1(input_ids_1, token_weights_1)

# Pass the modified token embeddings to the BERT model for further processing
output_1 = bert_model(inputs_embeds=modified_embeddings_1)

# Get the final output from the BERT model
pooled_output_1 = output_1[1]

In [ ]:
# Tokenize the sentence
input_ids_2 = torch.tensor(tokenizer.encode(sentence2, add_special_tokens=True)).unsqueeze(0)

# Define the token weights
token_weights_2 = torch.tensor([1,1,1,1,1,5,1]).float()

# Define the custom embedding layer with an embedding size of 768 (default for BERT)
custom_embedding_2 = CustomEmbedding(768)

# Get the modified token embeddings from the custom embedding layer
modified_embeddings_2 = custom_embedding_2(input_ids_2, token_weights_2)

# Pass the modified token embeddings to the BERT model for further processing
output_2 = bert_model(inputs_embeds=modified_embeddings_2)

# Get the final output from the BERT model
pooled_output_2 = output_2[1]

In [ ]:
import torch.nn.functional as F

# Calculate the cosine similarity between the two pooled outputs
cosine_sim = F.cosine_similarity(pooled_output_1, pooled_output_2)

# Print the cosine similarity
print("Cosine similarity between the two sentences:", cosine_sim.item())

Cosine similarity between the two sentences: 0.6122373938560486


### Stanza with BERT

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 11.6 MB/s eta 0:00:00


In [ ]:
!pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.5/802.5 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=9452bd336a849c07ba9e8e013f1160717b8b5bfea71ae12919625d9e28a9656d
  Stored in directory: /root/.cache/pip/wheels/9a/b8/0f/f580817231cbf59f6ade9fd132ff60ada1de9f7dc85521f857
Successfully built emoji


In [ ]:
import stanza

stanza.download('en')  # Download the English models




INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


In [ ]:
nlp = stanza.Pipeline(lang='en', processors="tokenize,pos,lemma,depparse")

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


In [ ]:
import torch
from transformers import BertModel, BertTokenizer

# Load the pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


#### Concat Pooled layer with Additional Embeddings

In [ ]:
# Define an input sequence
input_text = "The quick brown fox jumped over the lazy dog."

# Tokenize the input sequence
tokens = tokenizer(input_text, return_tensors='pt')

# Send the tokens to the model to get embeddings
outputs = model(**tokens)

# Extract the last hidden state of the model as the pooled representation
pooled_output = outputs.pooler_output

# Print the shape of the pooled output
print(pooled_output.shape)

torch.Size([1, 768])


In [ ]:
tokens

{'input_ids': tensor([[  101,  1996,  4248,  2829,  4419,  5598,  2058,  1996, 13971,  3899,
          1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
# Get the embeddings for the word "lazy" using Stanza
nlp = stanza.Pipeline(lang='en', processors='tokenize,pos,lemma', tokenize_no_ssplit=True)

doc = nlp(input_text)



In [ ]:
# Use pre-trained word embeddings to generate fixed-length vector representations for each word
word_emb = []
for sent in dep_trees:
    emb = []
    for word in sent:
        emb.append(model.embeddings.word_embeddings(torch.tensor([tokenizer.convert_tokens_to_ids(word['text'])])))
    word_emb.append(torch.mean(torch.stack(emb), dim=1))
word_emb = torch.stack(word_emb)

In [ ]:
word_emb.shape

torch.Size([1, 5, 768])

In [ ]:
mean_x = torch.mean(word_emb, dim=1)
mean_x.shape


torch.Size([1, 768])

In [ ]:
# Concatenate the BERT embeddings with the Stanza embeddings for "lazy"
pooled_output = torch.cat((model(**tokens)[0][:, -1], mean_x), dim=0)

# Print the shape of the pooled output
print(pooled_output.shape)

torch.Size([2, 768])
